## Validate and Convert _P.verrucosa_ GFF to GTF

### Notebook relies on:

- [GffRead](https://github.com/gpertea/gffread)

### List computer specs

In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Mon Feb 20 12:04:26 PM PST 2023
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.1 LTS
Release:	22.04
Codename:	jammy

------------
HOSTNAME: 
computer

------------
Computer Specs:

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Address sizes:                   45 bits physical, 48 bits virtual
Byte Order:                      Little Endian
CPU(s):                          4
On-line CPU(s) list:             0-3
Vendor ID:                       GenuineIntel
Model name:                      Intel(R) Core(TM) i9-10885H CPU @ 2.40GHz
CPU family:                      6
Model:                           165
Thread(s) per core:              1
Core(s) per socket:              1
Socket(s):                       4
Stepping:                        2
BogoMIPS:                        4800.01
Flags:                           fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss syscall

No LSB modules are available.


### Set variables
- `%env` indicates a bash variable

- without `%env` is Python variable

In [14]:
# Set directories, input/output files
%env data_dir=/home/sam/data/M_capitata/genomes
%env analysis_dir=/home/sam/analyses/20230127-pver-gff_to_gtf
analysis_dir="20230127-pver-gff_to_gtf"

# Input files (from NCBI)
%env gff=Pver_genome_assembly_v1.0.gff3

# URL of file directory
%env url=https://owl.fish.washington.edu/halfshell/genomic-databank

# Output file(s)
%env valid_gff=Pver_genome_assembly_v1.0-valid.gff3
%env gtf=Pver_genome_assembly_v1.0-valid.gtf


# Set program locations
%env gffread=/home/sam/programs/gffread-0.12.7.Linux_x86_64/gffread

# Set some formatting stuff
%env break_line=--------------------------------------------------------------------------

env: data_dir=/home/sam/data/M_capitata/genomes
env: analysis_dir=/home/sam/analyses/20230127-pver-gff_to_gtf
env: gff=Pver_genome_assembly_v1.0.gff3
env: url=https://owl.fish.washington.edu/halfshell/genomic-databank
env: valid_gff=Pver_genome_assembly_v1.0-valid.gff3
env: gtf=Pver_genome_assembly_v1.0-valid.gtf
env: gffread=/home/sam/programs/gffread-0.12.7.Linux_x86_64/gffread
env: break_line=--------------------------------------------------------------------------


### Create analysis directory

In [3]:
%%bash
# Make analysis and data directory, if doesn't exist
mkdir --parents "${analysis_dir}"

mkdir --parents "${data_dir}"

### Download GFF

In [4]:
%%bash
cd "${data_dir}"

# Download with wget.
# Use --quiet option to prevent wget output from printing too many lines to notebook
# Use --continue to prevent re-downloading fie if it's already been downloaded.
# Use --no-check-certificate to avoid download error from gannet
wget --quiet \
--continue \
--no-check-certificate \
${url}/${gff}

ls -ltrh "${gff}"

-rw-rw-r-- 1 sam sam 71M Mar 23  2020 Pver_genome_assembly_v1.0.gff3


### Examine GFF

#### Check first 20 lines

In [5]:
%%bash
head -n 20 "${data_dir}"/"${gff}"

# ORIGINAL: Pver_g1.t2 original gene structure, not modified by PASA
Pver_Sc0000000_size2095917	.	gene	13766	20466	.	+	.	ID=Pver_gene_g1;Name=Pver_g1.t1
Pver_Sc0000000_size2095917	.	mRNA	13766	20466	.	+	.	ID=Pver_g1.t2;Parent=Pver_gene_g1;Name=Pver_g1.t1
Pver_Sc0000000_size2095917	.	five_prime_UTR	13766	14013	.	+	.	ID=Pver_g1.t2.utr5p1;Parent=Pver_g1.t2
Pver_Sc0000000_size2095917	.	exon	13766	14098	.	+	.	ID=Pver_g1.t2.exon1;Parent=Pver_g1.t2
Pver_Sc0000000_size2095917	.	CDS	14014	14098	.	+	0	ID=cds.Pver_g1.t2;Parent=Pver_g1.t2
Pver_Sc0000000_size2095917	.	exon	16629	16667	.	+	.	ID=Pver_g1.t2.exon2;Parent=Pver_g1.t2
Pver_Sc0000000_size2095917	.	CDS	16629	16667	.	+	1	ID=cds.Pver_g1.t2;Parent=Pver_g1.t2
Pver_Sc0000000_size2095917	.	exon	17615	17698	.	+	.	ID=Pver_g1.t2.exon3;Parent=Pver_g1.t2
Pver_Sc0000000_size2095917	.	CDS	17615	17698	.	+	1	ID=cds.Pver_g1.t2;Parent=Pver_g1.t2
Pver_Sc0000000_size2095917	.	exon	18109	18420	.	+	.	ID=Pver_g1.t2.exon4;Parent=Pver_g1.t2
Pver_Sc0000000_size2095

#### Count unique number of fields

This identifies if there are rows with >9 fields (which there shouldn't be in a [GFF3](http://gmod.org/wiki/GFF3)).

In [15]:
%%bash
# Capture number of fields (NF) in each row in array.
field_count_array=($(awk -F "\t" '{print NF}' "${data_dir}/${gff}" | sort --unique))

# Check array contents
echo "List of number of fields in ${data_dir}/${gff}:"
echo ""
for field_count in "${field_count_array[@]}"
do
  echo "${field_count}"
done

echo ""
echo "${break_line}"
echo ""

# Preview of each line "type" with a given number of fields
# Check array contents
echo ""
for field_count in "${field_count_array[@]}"
do
  echo "Preview of lines with ${field_count} field(s):"
  echo ""
  awk \
    -v field_count="${field_count}" \
    -F "\t" \
    'NF == field_count' \
    "${data_dir}/${gff}" \
    | head
  echo ""
  echo "${break_line}"
  echo ""
done


List of number of fields in /home/sam/data/M_capitata/genomes/Pver_genome_assembly_v1.0.gff3:

1
10
11
2
9

--------------------------------------------------------------------------


Preview of lines with 1 field(s):

# ORIGINAL: Pver_g1.t2 original gene structure, not modified by PASA
# PASA_UPDATE: Pver_g2.t1, single gene model update, valid-1, status:[pasa:asmbl_2,status:12], valid-1
# PASA_UPDATE: Pver_g3.t1, single gene model update, valid-1, status:[pasa:asmbl_3,status:8], valid-1
# PASA_UPDATE: Pver_g4.t1, single gene model update, valid-1, status:[pasa:asmbl_4,status:13], valid-1
# ORIGINAL: Pver_g5.t1 original gene structure, not modified by PASA
# PASA_UPDATE: Pver_g6.t1, single gene model update, valid-1, status:[pasa:asmbl_6,status:8], valid-1
# ORIGINAL: Pver_g7.t1 original gene structure, not modified by PASA
# PASA_UPDATE: Pver_g8.t1, single gene model update, valid-1, status:[pasa:asmbl_8,status:8], valid-1
# PASA_UPDATE: Pver_g9.t1, single gene model update, valid-1,

In the above results, we can see that there are rows with 10 and 11 fields, due to incorrect tabs entered in Field 9. This creates an invalid GFF3 file.

### Fix invalid tabs in Field 9

This will remove tabs in Field 9 and join those extra fields into Field 9 using a semi-colon separator with the `Note=` attribute.

In [19]:
%%bash
time \
while read -r line
do
    # Count number of fields
    number_of_fields=$(echo "${line}" | awk -F "\t" '{print NF}')

    # If number of fields is 9, print the entire line
    if [[ "${number_of_fields}" -eq 9 ]]; then
      echo "${line}"
    # If there are 10 fields, cut/capture the first 9 and
    # capture the 10th.
    # Use printf to join 10th field to 9th via semi-colon
    elif [[ "${number_of_fields}" -eq 10 ]]; then
      first_nine=$(echo "${line}" | cut -f 1-9)
      tenth=$(echo "${line}" | cut -f 10 )
      printf "%s;%s\n" "${first_nine}" "Note=${tenth}"
    # If there are 11 fields, cut/capture the first 9, followed
    # by capturing the 10th and 11th fields.
    # Use printf to join 10th and 11th field to 9th via semi-colon.
    elif [[ "${number_of_fields}" -eq 11 ]]; then
      first_nine=$(echo "${line}" | cut -f 1-9)
      tenth=$(echo "${line}" | cut -f 10)
      eleventh=$(echo "${line}" | cut -f 11)
      printf "%s;%s;%s\n" "${first_nine}" "Note=${tenth}" "Note=${eleventh}"
    else
      # If the line doesn't match any of the above, just print it.
      echo "${line}"
    fi

done < "${data_dir}/${gff}" > "${analysis_dir}/${valid_gff}"


real	63m22.307s
user	62m10.233s
sys	12m37.124s


#### Compare line numbers between the two GFFs to make sure we didn't lose anything

In [20]:
%%bash
wc -l "${data_dir}/${gff}"
wc -l "${analysis_dir}/${valid_gff}"

575514 /home/sam/data/M_capitata/genomes/Pver_genome_assembly_v1.0.gff3
575514 /home/sam/analyses/20230127-pver-gff_to_gtf/Pver_genome_assembly_v1.0-valid.gff3


#### Count fields to make sure no more rows with >9 fields

In [21]:
%%bash
awk -F "\t" '{print NF}' "${analysis_dir}/${valid_gff}" | sort --unique

1
2
9


### Convert GFF to GTF

In [22]:
%%bash
cd "${data_dir}"

${gffread} -E \
"${analysis_dir}/${valid_gff}" -T \
1> ${analysis_dir}/"${gtf}" \
2> ${analysis_dir}/gffread-gff_to_gtf.stderr

### Inspect GTF

In [23]:
%%bash
head ${analysis_dir}/"${gtf}"

Pver_Sc0000000_size2095917	.	transcript	13766	20466	.	+	.	transcript_id "Pver_g1.t2"; gene_id "Pver_gene_g1"
Pver_Sc0000000_size2095917	.	exon	13766	14098	.	+	.	transcript_id "Pver_g1.t2"; gene_id "Pver_gene_g1";
Pver_Sc0000000_size2095917	.	exon	16629	16667	.	+	.	transcript_id "Pver_g1.t2"; gene_id "Pver_gene_g1";
Pver_Sc0000000_size2095917	.	exon	17615	17698	.	+	.	transcript_id "Pver_g1.t2"; gene_id "Pver_gene_g1";
Pver_Sc0000000_size2095917	.	exon	18109	18420	.	+	.	transcript_id "Pver_g1.t2"; gene_id "Pver_gene_g1";
Pver_Sc0000000_size2095917	.	exon	18845	19071	.	+	.	transcript_id "Pver_g1.t2"; gene_id "Pver_gene_g1";
Pver_Sc0000000_size2095917	.	exon	19404	19581	.	+	.	transcript_id "Pver_g1.t2"; gene_id "Pver_gene_g1";
Pver_Sc0000000_size2095917	.	exon	19848	20466	.	+	.	transcript_id "Pver_g1.t2"; gene_id "Pver_gene_g1";
Pver_Sc0000000_size2095917	.	CDS	14014	14098	.	+	0	transcript_id "Pver_g1.t2"; gene_id "Pver_gene_g1";
Pver_Sc0000000_size2095917	.	CDS	16629	16667	.	+	2	transcrip

### Generate checksum(s)

In [24]:
%%bash
cd "${analysis_dir}"

for file in *
do
  md5sum "${file}" | tee --append checksums.md5
done

9828841097f80b93fc2c599b76432b7c  gffread-gff_to_gtf.stderr
5dd8f21a4faea1f46c48a5ab253749d7  Pver_genome_assembly_v1.0-valid.gff3
c3cc8fb576bcf39dd17b6d229100aa56  Pver_genome_assembly_v1.0-valid.gtf


### Document GffRead program options

In [25]:
%%bash
${gffread} -h

gffread v0.12.7. Usage:
gffread [-g <genomic_seqs_fasta> | <dir>] [-s <seq_info.fsize>] 
 [-o <outfile>] [-t <trackname>] [-r [<strand>]<chr>:<start>-<end> [-R]]
 [--jmatch <chr>:<start>-<end>] [--no-pseudo] 
 [-CTVNJMKQAFPGUBHZWTOLE] [-w <exons.fa>] [-x <cds.fa>] [-y <tr_cds.fa>]
 [-j ][--ids <IDs.lst> | --nids <IDs.lst>] [--attrs <attr-list>] [-i <maxintron>]
 [--stream] [--bed | --gtf | --tlf] [--table <attrlist>] [--sort-by <ref.lst>]
 [<input_gff>] 

 Filter, convert or cluster GFF/GTF/BED records, extract the sequence of
 transcripts (exon or CDS) and more.
 By default (i.e. without -O) only transcripts are processed, discarding any
 other non-transcript features. Default output is a simplified GFF3 with only
 the basic attributes.
 
Options:
 --ids discard records/transcripts if their IDs are not listed in <IDs.lst>
 --nids discard records/transcripts if their IDs are listed in <IDs.lst>
 -i   discard transcripts having an intron larger than <maxintron>
 -l   discard transcripts

CalledProcessError: Command 'b'${gffread} -h\n'' returned non-zero exit status 1.